In [1]:
import os
import pandas as pd
import numpy as np
from implicit.als import AlternatingLeastSquares

## 데이터 준비와 전처리

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
# 필요없는 타임스태프제거
use_columns = ["user_id", "movie_id", "counts"]
ratings = ratings[use_columns]
ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [6]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
# 검색을 쉽게 하기 위해 소문자로 변환
movies['title'] = movies['title'].str.lower()
movies['genre'] = movies['genre'].str.lower()

movies["genre"] = movies["genre"].str.replace("|", ",")

/tmp/ipykernel_255/2284540720.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movies["genre"] = movies["genre"].str.replace("|", ",")


In [9]:
movies.head()

,movie_id,title,genre
0,1,toy story (1995),"animation,children's,comedy"
1,2,jumanji (1995),"adventure,children's,fantasy"
2,3,grumpier old men (1995),"comedy,romance"
3,4,waiting to exhale (1995),"comedy,drama"
4,5,father of the bride part ii (1995),comedy


## 영화갯수, 사용자수, 인기있는영화 분석 

In [10]:
# id갯수확인
ratings['movie_id'].nunique()

3628

In [11]:
# user갯수확인
ratings['user_id'].nunique()

6039

In [12]:
ratings["movie_id"].isna().sum()

0

In [13]:
ratings["user_id"].isna().sum()

0

In [14]:
ratings["counts"].isna().sum()

0

In [15]:
movies["movie_id"].isna().sum()

0

In [16]:
movies["title"].isna().sum()

0

In [17]:
movies["genre"].isna().sum()

0

In [18]:
data = pd.merge(movies, ratings)
data

,movie_id,title,genre,user_id,counts
0,1,toy story (1995),"animation,children's,comedy",1,5
1,1,toy story (1995),"animation,children's,comedy",6,4
2,1,toy story (1995),"animation,children's,comedy",8,4
3,1,toy story (1995),"animation,children's,comedy",9,5
4,1,toy story (1995),"animation,children's,comedy",10,5
...,...,...,...,...,...
836473,3952,"contender, the (2000)","drama,thriller",5682,3
836474,3952,"contender, the (2000)","drama,thriller",5812,4
836475,3952,"contender, the (2000)","drama,thriller",5831,3
836476,3952,"contender, the (2000)","drama,thriller",5837,4


In [19]:
# 인기있는영화
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [20]:
# 리뷰비율
user_count = data.groupby("user_id")["title"].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [21]:
# 영확목록확인
moo = movies["title"]
mo = moo.tolist()
mo

['toy story (1995)',
 'jumanji (1995)',
 'grumpier old men (1995)',
 'waiting to exhale (1995)',
 'father of the bride part ii (1995)',
 'heat (1995)',
 'sabrina (1995)',
 'tom and huck (1995)',
 'sudden death (1995)',
 'goldeneye (1995)',
 'american president, the (1995)',
 'dracula: dead and loving it (1995)',
 'balto (1995)',
 'nixon (1995)',
 'cutthroat island (1995)',
 'casino (1995)',
 'sense and sensibility (1995)',
 'four rooms (1995)',
 'ace ventura: when nature calls (1995)',
 'money train (1995)',
 'get shorty (1995)',
 'copycat (1995)',
 'assassins (1995)',
 'powder (1995)',
 'leaving las vegas (1995)',
 'othello (1995)',
 'now and then (1995)',
 'persuasion (1995)',
 'city of lost children, the (1995)',
 'shanghai triad (yao a yao yao dao waipo qiao) (1995)',
 'dangerous minds (1995)',
 'twelve monkeys (1995)',
 'wings of courage (1995)',
 'babe (1995)',
 'carrington (1995)',
 'dead man walking (1995)',
 'across the sea of time (1995)',
 'it takes two (1995)',
 'clueless (

## 선호하는영화를 ratings에 추가

In [22]:
my_favorite = [ 'crimson tide (1995)', 'party girl (1995)', 'castle freak (1995)', 'gordy (1995)', 'boxing helena (1993)']

my_playlist = pd.DataFrame({'user_id': ['aiaia']*5, 'title': my_favorite, 'counts':[5]*5})

# 임의 데이터 aiaia추가
if not data.isin({"user_id":["aiaia"]})["user_id"].any():
    data = data.append(my_playlist)
    
data.tail(10)

,movie_id,title,genre,user_id,counts
836473,3952.0,"contender, the (2000)","drama,thriller",5682,3
836474,3952.0,"contender, the (2000)","drama,thriller",5812,4
836475,3952.0,"contender, the (2000)","drama,thriller",5831,3
836476,3952.0,"contender, the (2000)","drama,thriller",5837,4
836477,3952.0,"contender, the (2000)","drama,thriller",5998,4
0,NaN,crimson tide (1995),NaN,aiaia,5
1,NaN,party girl (1995),NaN,aiaia,5
2,NaN,castle freak (1995),NaN,aiaia,5
3,NaN,gordy (1995),NaN,aiaia,5
4,NaN,boxing helena (1993),NaN,aiaia,5


In [23]:
# movie id, genre삭제
data = data.drop(columns=["movie_id", "genre"])
data.tail(10)

,title,user_id,counts
836473,"contender, the (2000)",5682,3
836474,"contender, the (2000)",5812,4
836475,"contender, the (2000)",5831,3
836476,"contender, the (2000)",5837,4
836477,"contender, the (2000)",5998,4
0,crimson tide (1995),aiaia,5
1,party girl (1995),aiaia,5
2,castle freak (1995),aiaia,5
3,gordy (1995),aiaia,5
4,boxing helena (1993),aiaia,5


In [24]:
# 고유한 유저, title을 찾아내기
user_unique = data["user_id"].unique()
title_unique = data["title"].unique()

# 유저, 타이틀 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [25]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['aiaia'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(title_to_idx['crimson tide (1995)'])

6039
152


In [26]:
# indexing을 통해 데이터 컬럼 내 값을 바꿈
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구하기
# 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거 

temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체
else:
    print('user_id column indexing Fail!!')

# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,title,user_id,counts
0,0,0,5
1,0,1,4
2,0,2,4
3,0,3,5
4,0,4,5
...,...,...,...
0,152,6039,5
1,178,6039,5
2,211,6039,5
3,232,6039,5


## CSR matrix 생성

In [27]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## als_model = AlternatingLeastSquares 모델을  구성하여 훈련

In [28]:
from implicit.als import AlternatingLeastSquares

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [29]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [30]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [31]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

In [57]:
aiaia, ghostbusters = user_to_idx["aiaia"], title_to_idx["ghostbusters (1984)"]
aiaia_vector, ghostbusters_vector = als_model.user_factors[aiaia], als_model.item_factors[ghostbusters]
aiaia_vector

array([-0.14237739, -0.16029784, -0.6002037 , -0.21052162, -0.01554354,
       -0.07295868, -0.14327073,  0.04185092, -0.11205214, -0.18116528,
        0.34670016, -0.52898455,  0.44124913, -0.04894805, -0.08312479,
        0.10971969, -0.4035848 ,  0.37188977,  0.18157932,  0.47353208,
        0.28506926, -0.06165987, -0.1322446 ,  0.45931   , -0.18546748,
       -0.07789388,  0.17581205, -0.28174773, -0.27532494,  0.10533485,
       -0.2907542 , -0.19176015, -0.48938558, -0.02249003,  0.1315479 ,
       -0.20480706,  0.43908635, -0.24820073,  0.06429934,  0.20953807,
        0.2861767 , -0.08644548,  0.20654963,  0.17186157,  0.3335689 ,
        0.2718221 , -0.22820094, -0.12216207, -0.30407768,  0.28191245,
       -0.195898  , -0.01123318, -0.35742638,  0.04857465,  0.08532545,
       -0.10544276,  0.3105838 , -0.19270182, -0.08281101, -0.11012191,
        0.4990927 , -0.0385053 ,  0.12742619,  0.19593921,  0.15616585,
       -0.22467043,  0.06565552,  0.6346438 ,  0.18198147,  0.25

In [58]:
ghostbusters_vector

array([-0.00661737,  0.02094847, -0.00447652,  0.01699581,  0.00783013,
       -0.00165443,  0.01724507,  0.01266383, -0.00832043, -0.01042304,
        0.00244228,  0.01193438,  0.04121414,  0.03996334,  0.00778342,
        0.02045357,  0.01764239,  0.02110928, -0.01066044,  0.01560415,
       -0.0098504 ,  0.01381438, -0.00234461, -0.00267835,  0.02689181,
        0.02478055,  0.04797271,  0.01356983,  0.00928116,  0.02664202,
       -0.01037539, -0.02406474,  0.01365163,  0.02307802,  0.00891634,
        0.02405449, -0.00440886, -0.00430602,  0.00261696,  0.00689903,
        0.02733607,  0.0130528 ,  0.004346  , -0.00830193,  0.01133651,
        0.02358017,  0.0082792 ,  0.0167752 ,  0.01095064, -0.03679392,
       -0.02727097,  0.02064172,  0.00682251, -0.00507115, -0.00855939,
        0.03073312, -0.01700777, -0.01854451, -0.00020808,  0.01645894,
       -0.00439682,  0.02704288, -0.00495036, -0.0119865 , -0.00146164,
       -0.00942068, -0.01983792, -0.01401954,  0.00483228, -0.01

## 내가선호하는영화와 비슷한 영화 추천

In [59]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [62]:
get_similar_title('crimson tide (1995)')

['crimson tide (1995)',
 'courage under fire (1996)',
 'three kings (1999)',
 'primal fear (1996)',
 'casino (1995)',
 'ransom (1996)',
 'rob roy (1995)',
 'copycat (1995)',
 'firm, the (1993)',
 'michael collins (1996)']

In [64]:
get_similar_title('party girl (1995)')

['party girl (1995)',
 'house of yes, the (1997)',
 'denise calls up (1995)',
 'kicking and screaming (1995)',
 'suburbia (1997)',
 'clockwatchers (1997)',
 'henry fool (1997)',
 'hotel de love (1996)',
 'spanking the monkey (1994)',
 'niagara, niagara (1997)']

In [63]:
get_similar_title('gordy (1995)')

['gordy (1995)',
 'gnome-mobile, the (1967)',
 'land before time iii: the time of the great giving (1995)',
 'zeus and roxanne (1997)',
 "we're back! a dinosaur's story (1993)",
 'air bud: golden receiver (1998)',
 'flipper (1996)',
 'that darn cat! (1997)',
 'fluke (1995)',
 'all dogs go to heaven 2 (1996)']

## 내가 가장 좋아할 만한 영화 추천

In [69]:
user = user_to_idx['aiaia']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(2632, 0.16022804),
 (336, 0.12659593),
 (604, 0.12655687),
 (617, 0.12229972),
 (15, 0.120593786),
 (1218, 0.1198105),
 (21, 0.11861148),
 (998, 0.112896174),
 (436, 0.11260775),
 (1278, 0.10464531),
 (509, 0.10089016),
 (996, 0.09992362),
 (1462, 0.09709049),
 (106, 0.09692102),
 (1599, 0.08891319),
 (1321, 0.08800936),
 (3435, 0.08460072),
 (764, 0.084301665),
 (463, 0.08412574),
 (1124, 0.0815042)]

In [70]:
[idx_to_title[i[0]] for i in artist_recommended]

['three kings (1999)',
 'client, the (1994)',
 'primal fear (1996)',
 'courage under fire (1996)',
 'casino (1995)',
 'cape fear (1991)',
 'copycat (1995)',
 'basic instinct (1992)',
 'firm, the (1993)',
 'last of the mohicans, the (1992)',
 "schindler's list (1993)",
 'platoon (1986)',
 'game, the (1997)',
 'braveheart (1995)',
 'wild things (1998)',
 'absolute power (1997)',
 'patriot, the (2000)',
 'ransom (1996)',
 'kalifornia (1993)',
 'glory (1989)']

In [79]:
ki = title_to_idx['three kings (1999)']
explain = als_model.explain(user, csr_data, itemid=ki)

In [81]:
# 선호한 5가지의 영화의 추천기여도확인
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('crimson tide (1995)', 0.1490100541103333),
 ('party girl (1995)', 0.010753098704212464),
 ('boxing helena (1993)', 0.0006956110275942052),
 ('castle freak (1995)', -0.0007677613209069019),
 ('gordy (1995)', -0.0009371827382217664)]